In [ ]:
import os

import sys
sys.path.insert(0, '..')

from sequencing_process.plot_fastq_gz_or_bam import plot_fastq_gz_or_bam
from sequencing_process.process_fastq_gz import align_fastq_gzs_using_bwa
from sequencing_process.process_bam import call_variants_on_bam_using_freebayes_and_multiprocess
from sequencing_process.process_vcf_gz import annotate_vcf_gz_using_snpeff, annotate_vcf_gz_using_bcftools, filter_vcf_gz_using_bcftools

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
GRCH_DIRECTORY_PATH = '/media/k/Chocolate/Jumis/data/grch'
assert os.path.isdir(GRCH_DIRECTORY_PATH)

# make_reference_genome_for_sequencing_process(GRCH_DIRECTORY_PATH)

FASTA_FILE_PATH = os.path.join(
    GRCH_DIRECTORY_PATH,
    'GCA_000001405.15_GRCh38_full_plus_hs38DH-extra_analysis_set.fa')
assert os.path.isfile(FASTA_FILE_PATH)

FASTA_GZ_FILE_PATH = FASTA_FILE_PATH + '.gz'
assert os.path.isfile(FASTA_GZ_FILE_PATH)

CLINVAR_VCF_GZ_FILE_PATH = os.path.join(GRCH_DIRECTORY_PATH, 'clinvar.vcf.gz')
assert os.path.isfile(CLINVAR_VCF_GZ_FILE_PATH)

In [ ]:
N_JOBS = 7
OVERWRITE = True

In [ ]:
FASTQ_GZ_0_FILE_PATH = '../../../person/genome.0.fastq.gz'
FASTQ_GZ_1_FILE_PATH = '../../../person/genome.1.fastq.gz'

plot_fastq_gz_or_bam(FASTQ_GZ_0_FILE_PATH, overwrite=OVERWRITE)
plot_fastq_gz_or_bam(FASTQ_GZ_1_FILE_PATH, overwrite=OVERWRITE)

In [ ]:
BAM_FILE_PATH = None

if not BAM_FILE_PATH:

    BAM_FILE_PATH = align_fastq_gzs_using_bwa(
        FASTA_GZ_FILE_PATH, [
            FASTQ_GZ_0_FILE_PATH,
            FASTQ_GZ_1_FILE_PATH,
        ],
        n_jobs=N_JOBS,
        overwrite=OVERWRITE)
    
    plot_fastq_gz_or_bam(BAM_FILE_PATH, overwrite=OVERWRITE)

In [ ]:
VCF_GZ_FILE_PATH = None

if not VCF_GZ_FILE_PATH:

    VCF_GZ_FILE_PATH = call_variants_on_bam_using_freebayes_and_multiprocess(
        BAM_FILE_PATH,
        FASTA_FILE_PATH,
        n_jobs=N_JOBS,
        overwrite=OVERWRITE)

    VCF_GZ_FILE_PATH = annotate_vcf_gz_using_snpeff(
        VCF_GZ_FILE_PATH, n_jobs=N_JOBS, overwrite=OVERWRITE)

    VCF_GZ_FILE_PATH = annotate_vcf_gz_using_bcftools(
        VCF_GZ_FILE_PATH,
        CLINVAR_VCF_GZ_FILE_PATH,
        '--columns =ID,INFO',
        n_jobs=N_JOBS,
        overwrite=OVERWRITE)

    VCF_GZ_FILE_PATH = filter_vcf_gz_using_bcftools(
        VCF_GZ_FILE_PATH, n_jobs=N_JOBS, overwrite=OVERWRITE)